In [82]:
import pandas as pd 
import numpy as np 

In [83]:
df_ratp = pd.read_csv('sanitaires-reseau-ratp.csv', sep=';')
df_paris = pd.read_csv('sanisettesparis.csv', sep=';')

In [84]:
df_ratp.head()

,Ligne,Station,Accessible au public,Tarif [Gratuit|Payant],Acces Passe Navigo ou Ticket T+,Acces Bouton poussoir,En zone controlee,Hors zone controlee station,Hors zone controlee voie publique,Accessibilite PMR,Localisation,coord_geo,gestionnaire
0,4,Bagneux Lucie Aubrac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Il est accessible depuis le guichet au niveau ...,"48.80357,2.3174",Toilette publique RATP
1,10,Cluny - La Sorbonne,oui,payant,oui,NaN,NaN,NaN,NaN,oui,"Sortie 2 boulevard St-Germain, à proximité du ...","48.85079575570429,2.3450308883652258",Toilette publique RATP
2,12,Concorde,non,gratuit,NaN,oui,oui,NaN,NaN,oui,"En direction de la ligne 1, vers la sortie 2, ...","48.866999231212986,2.32113887",Toilette publique RATP
3,14,Madeleine,oui,gratuit,NaN,oui,NaN,oui,NaN,oui,Sortie 1 dans la salle d'échange.,"48.869956563045214,2.3253476474582184",Toilette publique RATP
4,A,Charles de Gaulle - Etoile,oui,gratuit,NaN,oui,oui,NaN,NaN,non,"Sortie avenue Carnot, après la ligne de valide...","48.8740658,2.2951493",Toilette publique RATP


In [85]:
df_paris.head()

,TYPE,STATUT,ADRESSE,ARRONDISSEMENT,HORAIRE,ACCES_PMR,RELAIS_BEBE,URL_FICHE_EQUIPEMENT,geo_shape,geo_point_2d
0,SANISETTE,NaN,92102 RUE BLOMET,75015.0,6 h - 22 h,Oui,NaN,NaN,"{""coordinates"": [[2.300280980817689, 48.840905...","48.84090598651907,2.300280980817689"
1,SANISETTE,NaN,29 RUE MANIN,75019.0,24 h / 24,Oui,NaN,NaN,"{""coordinates"": [[2.3782622313212527, 48.87861...","48.87861935487687,2.3782622313212527"
2,SANISETTE,NaN,RUE DE CRIMEE,75019.0,6 h - 1 h,Oui,NaN,NaN,"{""coordinates"": [[2.3790339650916446, 48.88883...","48.88883561898796,2.3790339650916446"
3,SANISETTE,NaN,213 RUE DE BELLEVILLE,75019.0,6 h - 22 h,Oui,NaN,NaN,"{""coordinates"": [[2.394776616457134, 48.875445...","48.87544598661789,2.394776616457134"
4,SANISETTE,NaN,64 RUE DU VOLGA,75020.0,6 h - 22 h,Oui,NaN,NaN,"{""coordinates"": [[2.4089887074025595, 48.85271...","48.85271966367163,2.4089887074025595"


## Gestion des variables

les variables qui semblent les plus importantes sont évidemment les coordonnées géorgraphiques pour l'affichage sur la carte via Folium, le gestionnaire peut être intéressant pour savoir s'il s'agit d'une toilette en gare ou d'une toilette accessible sur la rue. Cette variable n'existe pas dans le df des sanisettes de Paris, on peut en conclure que c'est Paris elle-même qui les gère et donc le rajouter directement. 

L'indication quant à la localisation est importante mais non exploitable ici.
Les horaires sont importantes, on peut estimer que celles  accessibles en gare ont les mêmes horaires que les gares elles-mêms

L'information de l'accès PMR est indispensable, les valeurs manquantes seront remplacés par un "?". Lors de la filtration il sera jugé qu'elles ne sont pas accessibles pour les PMR. 
L'adresse n'est pas importante compte tenue des données géographiques et l'indication précise sur la carte

## Enlever les variables superflues

In [86]:
index_to_drop = []
for i in range(len(df_ratp)):
    if (df_ratp['Accessible au public'].iloc[i] != 'oui'):
        index_to_drop.append(i)

df_ratp = df_ratp.drop(index=index_to_drop, axis=0)

In [87]:
df_paris['ARRONDISSEMENT'] = df_paris['ARRONDISSEMENT'].astype(str)
df_paris['LOC'] = df_paris['ADRESSE']+' '+df_paris['ARRONDISSEMENT']

In [88]:
df_ratp['LOC'] = df_ratp['Station'] +' - '+df_ratp['Ligne']

In [89]:
df_ratp = df_ratp.drop(['Station', 'Ligne', 'Accessible au public', 'En zone controlee', 'Acces Passe Navigo ou Ticket T+', 'Acces Bouton poussoir',
                        'Hors zone controlee station', 'Hors zone controlee voie publique', 'Localisation'], axis=1)
df_paris = df_paris.drop(['STATUT','RELAIS_BEBE', 'URL_FICHE_EQUIPEMENT', 'geo_shape', 'ADRESSE', 'ARRONDISSEMENT','HORAIRE'], axis=1)


In [90]:
df_ratp.rename(columns={'Tarif [Gratuit|Payant]': 'TARIF', 'Accessibilite PMR': 'ACCES_PMR', 'gestionnaire':'GESTIONNAIRE'}, inplace=True)
df_paris.rename(columns={'geo_point_2d':'coord_geo'}, inplace=True)

In [91]:
df_ratp.columns

Index(['TARIF', 'ACCES_PMR', 'coord_geo', 'GESTIONNAIRE', 'LOC'], dtype='object')

In [92]:
df_ratp['TYPE'] = 'SANISETTE'
df_paris['GESTIONNAIRE'] = 'Paris'
df_paris['TARIF'] = 'gratuit'

## Merge

In [93]:
df = pd.concat([df_ratp, df_paris])
df

,TARIF,ACCES_PMR,coord_geo,GESTIONNAIRE,LOC,TYPE
1,payant,oui,"48.85079575570429,2.3450308883652258",Toilette publique RATP,Cluny - La Sorbonne - 10,SANISETTE
3,gratuit,oui,"48.869956563045214,2.3253476474582184",Toilette publique RATP,Madeleine - 14,SANISETTE
4,gratuit,non,"48.8740658,2.2951493",Toilette publique RATP,Charles de Gaulle - Etoile - A,SANISETTE
5,gratuit,oui,"48.891826754836515,2.237992043215619",Toilette publique RATP,La Defense (Grande Arche) - A,SANISETTE
6,gratuit,oui,"48.8484803444062,2.39676520957729",Toilette publique RATP,Nation - A,SANISETTE
...,...,...,...,...,...,...
610,gratuit,Oui,"48.83953772127824,2.4061405656535424",Paris,SQUARE CHARLES PEGUY 75012.0,TOILETTES
611,gratuit,Oui,"48.89555859860546,2.363815451695338",Paris,JARDIN RACHMANINOV 75018.0,TOILETTES
612,gratuit,Non,"48.833775430556955,2.3198918719148787",Paris,SQUARE DE LA ZAC DIDOT 75014.0,TOILETTES
613,gratuit,Oui,"48.865617429829534,2.2523240882300373",Paris,BOIS DE BOULOGNE - JARDIN DU PRE CATELAN 75016.0,TOILETTES


## Coordonnées

In [94]:
lat = []
long = []

for i in range(len(df)):
    coord = df['coord_geo'].iloc[i].split(',')
    lat.append(coord[0])
    long.append(coord[1])

df['LAT'] = lat
df['LONG'] = long
df = df.drop('coord_geo', axis=1)

In [95]:
df

,TARIF,ACCES_PMR,GESTIONNAIRE,LOC,TYPE,LAT,LONG
1,payant,oui,Toilette publique RATP,Cluny - La Sorbonne - 10,SANISETTE,48.85079575570429,2.3450308883652258
3,gratuit,oui,Toilette publique RATP,Madeleine - 14,SANISETTE,48.869956563045214,2.3253476474582184
4,gratuit,non,Toilette publique RATP,Charles de Gaulle - Etoile - A,SANISETTE,48.8740658,2.2951493
5,gratuit,oui,Toilette publique RATP,La Defense (Grande Arche) - A,SANISETTE,48.891826754836515,2.237992043215619
6,gratuit,oui,Toilette publique RATP,Nation - A,SANISETTE,48.8484803444062,2.39676520957729
...,...,...,...,...,...,...,...
610,gratuit,Oui,Paris,SQUARE CHARLES PEGUY 75012.0,TOILETTES,48.83953772127824,2.4061405656535424
611,gratuit,Oui,Paris,JARDIN RACHMANINOV 75018.0,TOILETTES,48.89555859860546,2.363815451695338
612,gratuit,Non,Paris,SQUARE DE LA ZAC DIDOT 75014.0,TOILETTES,48.833775430556955,2.3198918719148787
613,gratuit,Oui,Paris,BOIS DE BOULOGNE - JARDIN DU PRE CATELAN 75016.0,TOILETTES,48.865617429829534,2.2523240882300373


In [97]:
df.to_csv('paris_sanisette.csv', index=False)